# Midterm 2 Assignment 3 Davide Amadei

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from typing import Iterator

In [ ]:
def show_img(image: np.ndarray, gray: bool = True, title: str = None) -> None:
    """simple utility to show images with axes disabled

    Parameters
    ----------
    image : np.ndarray
        image to show
    gray : bool, optional
        if true show image in grayscale, by default True
    """
    if gray:
        plt.imshow(image, cmap="gray")
    else:
        plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.axis("off")
    plt.show()

## Function to read MNIST dataset

In [ ]:
def read_MNIST_file(path: str, reshape: bool = False):
    with open(path, "rb") as f:
        magic = int.from_bytes(f.read(4))
        n_items = int.from_bytes(f.read(4))
        #image file
        if magic == 2051:
            n_rows = int.from_bytes(f.read(4))
            n_columns = int.from_bytes(f.read(4))
            if reshape:
                shape = (n_items, n_rows, n_columns)
            else:
                shape = (n_items, n_rows * n_columns)
            images = np.ndarray(shape)
            for i in range(n_items):
                img_buffer = f.read(n_rows*n_columns)
                img = np.frombuffer(img_buffer, np.uint8)
                if reshape:
                    img = np.reshape(img, (n_rows, n_columns))
                images[i] = img
            return (shape, images/255)
        #label file
        elif magic == 2049:
            pass
        else:
            raise ValueError("wrong file format")

In [ ]:
tr_img_shape, tr_images = read_MNIST_file("./data/train-images-idx3-ubyte")
ts_img_shape, ts_images = read_MNIST_file("./data/t10k-images-idx3-ubyte")

## Restricted Boltzmann Machine implementation

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

class RBM:
    def __init__(self, v_units: int, h_units:int, seed:int = None):
        self._rng = np.random.default_rng(seed)
        self._seed = seed
        self._weights = self._rng.normal(size=(v_units, h_units))
        self._bv = self._rng.normal(size=v_units)
        self._bh = self._rng.normal(size=h_units)
        # self._bv = self._rng.uniform(0, 1, size=v_units)
        # self._bh = self._rng.uniform(0, 1, size=h_units)
    
    @staticmethod
    def get_minibatches(
        x: np.ndarray, batchsize: int
    ) -> Iterator[tuple[np.ndarray, np.ndarray]]:
        """Returns minibatches of given size over (x, y).

        Parameters
        ----------
        x : np.ndarray
            data array
        batchsize : int
            batch size of yielded minibatches

        Yields
        ------
        Iterator[tuple[np.ndarray,np.ndarray]]
            iterator over minibatches
        """
        if batchsize in [None, 0, -1]:
            batchsize = x.shape[0]
        size = x.shape[0]
        batchtotal, remainder = divmod(size, batchsize)
        for i in range(batchtotal):
            mini_x = x[i * batchsize : (i + 1) * batchsize]
            yield mini_x
        if remainder > 0:
            yield (x[batchtotal * batchsize :])

    def sample_data(self, data):
        return self._rng.binomial(1, p=data)
    
    def reset(self):
        self._rng = np.random.default_rng(self._seed)
        self._weights = self._rng.normal(size=self._weights.shape)
        self._bv = self._rng.normal(size=self._bv.shape)
        self._bh = self._rng.normal(size=self._bh.shape)
        
    def fit(self, tr_data, max_epochs=10, eta=1e-3, batchsize=128):
        for i in range(max_epochs):
            shuffled_data = tr_data[self._rng.permutation(tr_data.shape[0])]

            for batch in RBM.get_minibatches(shuffled_data, batchsize):
                
                sampled_data = self.sample_data(batch)
                h_prob0 = sigmoid(np.dot(sampled_data, self._weights) + self._bh)
                sampled_h0 = self.sample_data(h_prob0)

                wake = np.dot(sampled_data.T, sampled_h0)

                recon_data_prob = sigmoid(np.dot(sampled_h0, self._weights.T) + self._bv)
                recon_data = self.sample_data(recon_data_prob)

                h_prob1 = sigmoid(np.dot(recon_data, self._weights) + self._bh)
                # sampled_h1 = self.sample_data(h_prob1)
                
                dream = np.dot(recon_data.T, h_prob1)

                self._weights += eta * (wake - dream)/batch.shape[0]
                self._bv += eta * (np.sum(sampled_data, axis=0) - np.sum(recon_data, axis=0))/batch.shape[0]
                self._bh += eta * (np.sum(h_prob0, axis=0) - np.sum(h_prob1, axis=0))/batch.shape[0]
            print(f"Epoch number {i+1} done")
    
    def reconstruct(self, ts_data):
        sampled_data = self.sample_data(ts_data)

        h_prob0 = sigmoid(np.dot(sampled_data, self._weights) + self._bh)
        sampled_h0 = self.sample_data(h_prob0)
        
        recon_data_prob = sigmoid(np.dot(sampled_h0, self._weights.T) + self._bv)
        return recon_data_prob


In [ ]:
boltzmann_machine = RBM(tr_img_shape[1], 100)

In [ ]:
boltzmann_machine.reset()
boltzmann_machine.fit(tr_images, 20, 1, 100)

In [ ]:
reconstructed_test = np.reshape(boltzmann_machine.reconstruct(ts_images),
                                (ts_images.shape[0], 28, 28))
ts_images_reshaped = np.reshape(ts_images, (ts_images.shape[0], 28, 28))
sampled_recon_test = boltzmann_machine.sample_data(reconstructed_test)

In [ ]:
index = 352
show_img(ts_images_reshaped[index])
show_img(reconstructed_test[index])

In [ ]:
w_test = boltzmann_machine._weights
w_test = np.reshape(w_test, (28, 28, w_test.shape[1]))
show_img(w_test[:, :, 50])

In [ ]:
test = np.ones((20, 3))
lul = np.random.random(3)
print(lul)
print(test)
print(test - lul)
v = test - lul
c = test - np.broadcast_to(lul, test.shape)
print(v-c)

In [ ]:
test = np.random.random(10)
print(np.random.binomial(1, p=test))